In [466]:
import openpyxl
import pyomo.environ as pe
import numpy as np
import pandas as pd

t=17
power_limit = 700

In [467]:
#################################################################################################################
# Objetivo: Ler do excel "Info" informaçoes sobre cada load:
#
# ->Index: Indice de cada carga (1 a 15);
# ->Classificação em termos de flexibilidade: 0 (não flexivel), 1 (flexivel em potencia) e 2 (flexivel no tempo);
# ->T_ON: Tempo máximo que a carga deve estar ligada;
# ->T_OFF: Tempo máximo que a carga deve estar desligada;
# ->T_start: Modo no primeiro periodo: 0 (desligada) ou 1 (ligada);
# ->Power: Potência de cada carga quando está ligada.
#
#################################################################################################################

# load the workbook
workbook = openpyxl.load_workbook('Info.xlsx')

# select the worksheet
worksheet = workbook['Lds_inputs']

# read Index of cells
column = worksheet['B']
Index = []
for cell in column:
    if cell.row == 1:
        continue
    Index.append(cell.value)

# read Classification of cells
column = worksheet['C']
Class = []
for cell in column:
    if cell.row == 1:
        continue
    Class.append(cell.value)

# read T_ON of cells
column = worksheet['D']
T_ON = []
for cell in column:
    if cell.row == 1:
        continue
    T_ON.append(cell.value)

# read T_OFF of cells
column = worksheet['E']
T_OFF = []
for cell in column:
    if cell.row == 1:
        continue
    T_OFF.append(cell.value)

# read T_start of cells
column = worksheet['F']
T_start = []
for cell in column:
    if cell.row == 1:
        continue
    T_start.append(cell.value)

# read Power
column = worksheet['G']
Power = []
for cell in column:
    if cell.row == 1:
        continue
    Power.append(cell.value)

# print the values
print(f"Index - {(Index)}")
print(f"Classificação em termos de flexibilidade - {(Class)}")
print(f"T_ON - {(T_ON)}")
print(f"T_OFF - {(T_OFF)}")
print(f"Modo de começo - {(T_start)}")
print(f"Potência - {(Power)}")



Index - [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Classificação em termos de flexibilidade - [2, 2, 2, 1, 1, 1, 2, 2, 2, 0, 2, 2, 0, 0, 2]
T_ON - [3, 8, 4, 40, 40, 40, 15, 6, 38, 50, 7, 10, 3, 10, 32]
T_OFF - [4, 4, 10, 0, 0, 0, 4, 4, 65, 0, 45, 40, 0, 0, 60]
Modo de começo - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Potência - [90, 90, 90, 10, 10, 10, 590, 500, 180, 192, 300, 300, 1000, 2, 2000]


In [468]:
#Declaração de variáveis:

rows = t
cols = len(Index)

ON_OFF_vector_RT = [0] * len(Index)
ON_OFF_matrix = [['' for j in range(cols)] for i in range(rows)]
ON_OFF_before_optim = [['' for j in range(cols)] for i in range(rows)]
conter_on_dataset = [0] * len(Index)
cont_off_dataset = [0] * len(Index)
state = [0] * len(Index) #(0=desligado, 1=ligado)
T_ON_Original = [0] * len(Index)
T_ON_Original = T_ON.copy()

weight_vector_RT = [10] * len(Index)
weight_matrix = [[0 for j in range(cols)] for i in range(rows)]
weight_vector_before_optim = [['' for j in range(cols)] for i in range(rows)]
cont_on_weight = [0] * len(Index)
cont_off_weight = [0] * len(Index)
actual_weight__Operation_Out_Range = [0] * len(Index)
aux_optim = 0

vector_x = [0] * len(Index)
optimization =0

sum_Power=[0] * t
sum_optim=[0] * t

In [469]:

###########################################################################################################
# Objetivo: Cria matrix ON/OFF de acordo com as especificaçoes do excel Info (lido anteriormente),
#           irá representar um dataset de entrada.
#
#           -> Cargas flexiveis no tempo (index=1,2,3,7,8,9,11,12,15): ligam e desligam alternadamente de acordo 
#              com T_ON e T_OFF;
#           -> Cargas flexiveis em potencia (index=4,5,6): uma vez que não faz sentido terem um T_OFF, 
#              eu assumo que ligam às 07:00 , 19:00 , 20:00 e 21:00, ficando ligadas de acordo com T_ON;
#           -> Cargas não flexiveis (index=10,13,14): São consideradas Fixed Loads, assumo que são ligadas 
#              às 13:00 e 20:00, permanendo ligadas até T_ON.
#
# Variaveis: ON_or_OFF (0=OFF, 1=ON).
###########################################################################################################

def function_dataset(i, optimization, ON_OFF_before_optim, ON_OFF_matrix, ON_OFF_vector_RT):

    if (optimization==0):
        for j in range(len(Index)): 
            if (i==0):
                if (T_start[j] == 1):           #Se começam ON:
                    ON_OFF_vector_RT[j] = 1
                    state[j]= 1
            
                elif (T_start[j] == 0):         #Se começam OFF:
                    ON_OFF_vector_RT[j] = 0
                    state[j]= 0

            if i!=0 and Class[j]==2 and weight_matrix [i-1][j] == 20:
                state[j] = 1
                conter_on_dataset[j] = 0

            if i!=0 and Class[j] ==2 and weight_matrix [i-1][j]== -1:
                T_ON[j] = round(( (T_ON[j] * (T_OFF[j]+1)) / T_OFF[j] ))
                state[j] = 1
                conter_on_dataset[j] = 0
                #print(f'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA i: {i} , j: {j}, T_ON: {T_ON[j]}, Counter ON: {conter_on_dataset[j]} \n') 

            #if (i!=0 and Class[j] ==2) and T_ON[j] != T_ON_Original [j] and conter_on_dataset[j] <= T_ON[j]:
            #    state[j]=1

            


        for j in range(len(Index)):
        
            #Cargas flexiveis em tempo:
            if (Class[j]==2):
                if state[j] == 1:                              
                    if conter_on_dataset[j] < T_ON[j]:                    
                        ON_OFF_vector_RT[j] = 1
                        conter_on_dataset[j] += 1
                        state[j]=1
                    else:                                       
                        conter_on_dataset[j] = 0 
                        ON_OFF_vector_RT[j] = 0
                        cont_off_dataset[j] = 1
                        state[j]=0
                        T_ON[j] = T_ON_Original[j]

                elif state[j] == 0: 
                    #print(f'Cnt OFF: {cont_off_dataset[j]}, T_OFF: {T_OFF[j]}, j: {j}')                           
                    if cont_off_dataset [j] < T_OFF[j]:                 
                        ON_OFF_vector_RT[j] = 0
                        cont_off_dataset[j] += 1
                        state[j]=0
                    else:                                       
                        cont_off_dataset[j] = 0 
                        ON_OFF_vector_RT[j] = 1
                        conter_on_dataset[j] = 1
                        state[j]=1
    
            #Cargas flexiveis em potência ou não flexiveis:
            elif (Class[j]== 0 or Class[j]== 1):
                if state[j] == 1:                              
                    if conter_on_dataset[j] < T_ON[j]:                    
                        ON_OFF_vector_RT[j] = 1
                        conter_on_dataset[j] += 1
                        state[j]=1
                    else:                                       #
                        conter_on_dataset[j] = 0
                        ON_OFF_vector_RT[j] = 0
                        cont_off_dataset[j] = 1
                        state[j]=0

                elif state[j] == 0 :                          
                    if ((Class[j]== 1) and (i==7 or i==19*60 or i==20*60 or i==21*60)) or ((Class[j]== 0) and (i==13*60 or i==20*60)): 
                    #Ligar cargas com flex. em potencia (luzes) às 07:00 , 19:00 , 20:00 e 21:00; 
                    #Ligar cargas não flexiveis as 13:00 e 20:00;
                        ON_OFF_vector_RT[j] = 1
                        conter_on_dataset[j] += 1
                        state[j] = 1
                    else:
                        ON_OFF_vector_RT[j] = 0
                        state[j] = 0
                        conter_on_dataset[j] = 0
                        cont_off_dataset[j] = 1

        return (ON_OFF_vector_RT)


    elif (optimization==1):
        for j in range(len(Index)):  

            if (Class[j] == 2 and ON_OFF_vector_RT[j] == 0 and ON_OFF_before_optim[i][j] == 1):
                if ON_OFF_matrix[i-1][j] == 1 and ON_OFF_matrix[i-2][j] == 0:
                    state[j]=1
                    cont_off_dataset[j] = 0 
                    conter_on_dataset[j] = 1

                if ON_OFF_matrix[i-1][j] == 0 and ON_OFF_matrix[i-2][j] == 0:
                    state[j]=1
                    cont_off_dataset[j] = 0 
                    conter_on_dataset[j] = 1
                
                else:
                    state[j]=0
                    cont_off_dataset[j] = 1 
                    conter_on_dataset[j] = 0


In [470]:
###########################################################################################################
# Objetivo: Está dividido em duas partes:
#           1) Faz optimization dos weight_auxs apos a optmização;
#           2) Calcula os weight_auxs para cada carga antes da otimização;
#          
#           Os weight_auxs são de 0 a 10 e medem o comforto.
#           Caso a carga esteja OFF, os weight_auxs vão de 1 a 10 (prioridade aumenta, deve-se ligar a carga);
#           Caso a carga esteja ON, os weight_auxs vão de 10 a 1 (prioridade diminui, pode-se desligar a carga);
#           Em relação aos optimizations, o vetor "compare" serve para indicar quais das cargas foram alteradas;
#
# Nota: Em tempo real, apenas queremos cortar as cargas que estão ON com prioridades próximas de 1;
#       Deste modo, os weight_auxs calculados quando as cargas estão OFF não são relevantes;
#       No entanto, os weight_auxs foram calculados na mesma para cargas OFF, pois vai dar jeito para Hour-Ahead;
#       
# Variáveis: weight_aux [][]     
###########################################################################################################

def function_weight (i, ON_OFF_vector_RT, optimization, weight_vector_before_optim, ON_OFF_matrix, weight_matrix):
    weight_aux =[0] * len(Index)
    aux_optim = 0

    #Calculo dos weight_auxs antes da optimização
    if optimization == 0:
       
        for j in range(len(Index)): 
            if (i==0 and T_start[j] == 1 and Class[j] == 2):           #Se começam ON:
                weight_aux[j] = 10 - (9/T_ON[j] )
            
        
            if (i==0 and T_start[j] == 0 and Class[j] == 2):         #Se começam OFF:
                weight_aux[j] = 1 + (9/T_OFF[j])
   
            
            if Class[j] == 0 or Class[j] == 1:
                weight_aux[j] = -1
               
            """
            if (i!=0 and Class[j] ==2) and (weight_matrix [i-1][j]== -1  ):
                T_ON[j] = T_ON[j] + ( (T_ON[j] * (T_OFF[j]+1)) / T_OFF[j] )

            if (i!=0 and Class[j] ==2) and T_ON[j] == T_ON[j] + ( (T_ON[j] * (T_OFF[j]+1)) / T_OFF[j]) and conter_on_dataset[j] < T_ON[j]:
                ON_OFF_vector_RT[j] = 1
                conter_on_dataset[j] += 1
                state[j]=1

            else:
                T_ON[j] = T_ON[j]
            """
        
        for j in range(len(Index)):  
            
            if Class[j] ==2 and i!=0:
               

                if (weight_matrix [i-1][j] == 20 or weight_matrix [i-1][j] == -1):
                    weight_matrix [i-1][j] = actual_weight__Operation_Out_Range [j]
                    aux_optim =1 
                    #if (j==0):
                        #print(f"BBBBBBBBBBBBBBBBBBBBBBB weight_matrix {i}: {(weight_matrix[i][j])}\n ")


                if ON_OFF_vector_RT[j] == 1 :                                                                                               
                    weight_aux[j] =  weight_matrix [i-1][j] - (9/T_ON[j] )
                    if j==0:
                        print(f"i {i}, ")
                        print(f"ON OFF = 1, weight_aux {j}: {(weight_aux[j])}\n ")

                    if weight_aux[j] < 1:
                        weight_aux[j] = 1           
            
          
                if ON_OFF_vector_RT[j] == 0 :  
                    #print(f"weight_aux {j+1}: {(weight_aux[j])}\n ")
                    #print(f"weight_matrix {i+1}{j}: {(weight_matrix[i-1][j])}\n ")
                    #print(f"9/T_OFF {j+1}: {(9/T_OFF[j])}\n ")  
                    weight_aux[j] =  weight_matrix [i-1][j] + (9/T_OFF[j] )

                    if weight_aux[j] > 10 and aux_optim == 0:
                        weight_aux[j] = 10

                

            elif Class[j] == 0 or Class[j] == 1:
                weight_aux[j] = -1


        for j in range(len(Index)):
            weight_aux[j]=round(weight_aux[j], 3)
            
        return (weight_aux)


    #optimization dos weight_auxs apos a optimização
    elif optimization == 1:
        for j in range(len(Index)): 

            weight_aux[j] = weight_vector_before_optim[i][j]

            if Class[j] == 2 and ON_OFF_vector_RT[j]==0 and ON_OFF_before_optim[i][j]==1 :                                      
                if ON_OFF_matrix[i-1][j] == 1 and ON_OFF_matrix[i-2][j] == 0:
                    actual_weight__Operation_Out_Range [j] = weight_matrix[i-1][j] + 9/T_OFF[j]
                    weight_aux[j] = 20
                    if j==0:
                        print(f"i {i}, ")
                        print(f"peso= 20, weight_aux {j}: {(weight_aux[j])}\n ")
        

                elif ON_OFF_matrix[i-1][j] == 0 and ON_OFF_matrix[i-2][j] == 1 and ON_OFF_matrix[i-3][j] == 0:
                    actual_weight__Operation_Out_Range [j] = weight_matrix[i-1][j] + 9/T_OFF[j]
                    weight_aux[j] = -1
                    if j==0:
                        print(f"i {i}, ")
                        print(f"peso=-1, weight_aux {j}: {(weight_aux[j])}\n ")
                    
                    
                
                elif ON_OFF_matrix[i-1][j] == 0 and ON_OFF_matrix[i-2][j] == 0:
                    actual_weight__Operation_Out_Range [j] = weight_matrix[i-1][j] + 9/T_OFF[j]
                    weight_aux[j] = -1

                else:
                    weight_aux[j] = weight_matrix[i-1][j] + 9/T_OFF[j]
                    
                    

            elif Class[j] == 0 or Class[j] == 1:
                weight_aux[j] = -1
            
                                                          
       
        return(weight_aux)
             

   



In [471]:
###############################################################################
# Objetivo: Para cada periodo, sum a potência das cargas ON.
###############################################################################

def function_sum_Power (i, ON_OFF_vector_RT):
    
    aux=0
    sum_Power_aux=0
    sum_Power=[0] * t
    
    for j in range(len(Index)):     
        if ON_OFF_vector_RT[j] == 1:
            aux=Power[j]
            sum_Power_aux=sum_Power_aux + aux
    sum_Power [i]=sum_Power_aux
    sum_Power_aux=0
    
    return(sum_Power[i])





In [472]:
##########################################################################################
# Optimização: Maximiza o conforto (weight_auxs).
# Constraint: A sum de potência para cada periodo tem de ser menor do que a Potência max.
##########################################################################################

weight_vector_RT = [10] * len(Index)
ON_OFF = [0] * len(Index)
model = pe.ConcreteModel()

#Declaration
model.loads = pe.Set(initialize=np.arange(0,15))
model.limite = pe.Param(initialize=power_limit)
model.Power = pe.Param(range(len(Index)),initialize=Power)
model.weight_aux =pe.Param(range(len(Index)),initialize=weight_vector_RT,mutable=True)
model.y =pe.Param(range(len(Index)),initialize=ON_OFF,mutable=True)
model.x = pe.Var(range(len(Index)),domain=pe.Binary, initialize={i: 1 for i in range(len(Index))})

"""
# Prints de verificação
for i in range(0, 15):
    print(f"Index {i+1}: Power {i+1} = {(model.Power[i])}, weight_aux = {(model.weight_aux[i])} ")

print({pe.value(model.limite)})

for i in range(len(Index)):
    print(f" x[{i}] = {(model.x[i].value)}")


for i in range(len(Index)):
    print(model.Power[i])
    print(sum(model.Power[i]*model.x[i] for i in range(len(Index))))

    """

#Constrait 3
def _constraint_four(m):
   return sum((m.Power[j]*m.x[j]) for j in range(len(Index))) <= m.limite
model.constraint_four = pe.Constraint(rule=_constraint_four)

#Objective function
def _objective_function(m):
    return sum(m.weight_aux[j]*m.x[j] for j in range(len(Index)))
model.objective = pe.Objective(expr=_objective_function, sense=pe.maximize)

#vector_x = model.x()
#weight_aux_max = model.objective() 
#print(vector_x)
#print(weight_aux_max)

In [473]:
###########################################################################################################
# Objetivo: Resolve problema em Real-Time
# Funções usadas: 
#                   ->function_weight (dá weight_auxs e faz optimizations dos mesmos);
#                   ->function_sum (sum as potencias de cada carga para ti);
#                   ->optimização (decide quais cargas estão ON ou OFF, maximizando o conforto);
# Input: Informação de cada carga, dataset com ON/OFF.
# Output: Optimização do dataset.
###########################################################################################################


#Resolução do problema em Real-Time
for i in range (t): 
    optimization=0
    ON_OFF_vector_RT = function_dataset (i, optimization, ON_OFF_before_optim, ON_OFF_matrix, ON_OFF_vector_RT)
    ON_OFF_matrix[i] = ON_OFF_vector_RT.copy()
    
    weight_vector_RT = function_weight(i, ON_OFF_vector_RT, optimization, weight_vector_before_optim, ON_OFF_matrix, weight_matrix)             
    weight_matrix [i] = weight_vector_RT.copy()

    for j in range(len(Index)):                                                       
        model.weight_aux[j] = weight_vector_RT[j]
        model.y[j] = ON_OFF_vector_RT[j]

    def _constraint_three(m, j):
        if weight_vector_RT[j]<0: 
            return m.x[j] == m.y[j]
        else:
            return m.x[j] <= m.y[j]
    model.constraint_three = pe.Constraint(model.loads, rule=_constraint_three)

    sum_Power[i] = function_sum_Power(i, ON_OFF_vector_RT)                                             


    if sum_Power[i] > power_limit:                                                         
        results = pe.SolverFactory('scip', executable='C:/Program Files/SCIPOptSuite 8.0.3/bin/scip.exe').solve(model)
        #model.pprint()

        #print(f"\nPeriodo {i+1}: Resultados\n")
        for j in range(0, 15):
            #print(f"Index {j+1}: x[{j}] = {(model.x[j].value)}")
            #print(f"Index {j+1}: y[{j}] = {(model.y[j].value)}")
            #print(f"Index {j+1}: ON/OFF = {(ON_OFF_vector_RT[j])}\n")
            #print(f"weight_aux Max = {model.objective()}") 
            vector_x [j] = np.abs(model.x[j].value)                                    
            
        weight_aux_max = model.objective()                                                   

        optimization=1                                                                    
        ON_OFF_before_optim[i] = ON_OFF_vector_RT.copy()
        ON_OFF_vector_RT = vector_x.copy()
        ON_OFF_matrix[i] = ON_OFF_vector_RT.copy()
        function_dataset (i,optimization, ON_OFF_before_optim, ON_OFF_matrix, ON_OFF_vector_RT)                                                 
        
                                                                              
        weight_vector_before_optim[i] = weight_vector_RT.copy()                                                      
        weight_vector_RT = function_weight(i,ON_OFF_vector_RT,  optimization, weight_vector_before_optim, ON_OFF_matrix, weight_matrix)
        weight_matrix [i] = weight_vector_RT

        sum_optim[i] = function_sum_Power(i, ON_OFF_vector_RT)                                     
       

        


    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
i 4, 
ON OFF = 1, weight_aux 0: 7.0
 
    (type=<class 'pyomo.core.base.cons

In [474]:
##############################################################################
# Objetivo: ->Acrescentar os minutos e index á matrix ON_OFF e weight_vector_RTs;
#           ->Acrescentar a sum das potencia à matriz ON_or_OFF.
##############################################################################


col = []
for i in range(t):
    col.append(i)


for i in range(t):
    ON_OFF_matrix[i].insert(0, col[i])
    ON_OFF_before_optim[i].insert(0, col[i])
    weight_matrix[i].insert(0, col[i])
    weight_vector_before_optim[i].insert(0, col[i])
    

for i in range(t):
    ON_OFF_matrix[i].append(0)
    ON_OFF_matrix[i].append(0)
    #change value in ON_or_OFF list in position 16 for sum[i]
    ON_OFF_matrix[i][16] = sum_Power[i]
    ON_OFF_matrix[i][17] = sum_optim[i]
    

row = []
for i in range(len(Index)+3):
    row.append(i)
row [0] = 'Min/Index'
row [16] ='Sum Power'
row [17] ='Power Optim'
row1 = []
for i in range(len(Index)+1):
    row1.append(i)
row1 [0] = 'Min/Index'


ON_OFF_matrix.insert(0, row)
ON_OFF_before_optim.insert(0, row1)
weight_matrix.insert(0, row1)
weight_vector_before_optim.insert(0, row1)

In [475]:
############################################################################################################
# Objetivo: Imprime as 4 worksheets.
# Função: Escreve no excel: 
#               ->tabela com ON/OFF optimizado para cada load para ti;
#               ->tabela com ON/OFF antes da optimização;
#               ->tabela com weight_auxs optimizados;
#               ->tabela com weight_auxs antes da optimização;
############################################################################################################


# Create a Pandas Excel writer using XlsxWriter
writer = pd.ExcelWriter('Results.xlsx', engine='xlsxwriter')

df = pd.DataFrame(ON_OFF_matrix)
writer = pd.ExcelWriter('Results.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Dataset', index=False, header=False)

df2 = pd.DataFrame(ON_OFF_before_optim)
df2.to_excel(writer, sheet_name='Dataset before', index=False, header=False)

df3 = pd.DataFrame(weight_matrix)
df3.to_excel(writer, sheet_name='weight_vector_RT', index=False, header=False)

df4 = pd.DataFrame(weight_vector_before_optim)
df4.to_excel(writer, sheet_name='weight_vector_RT before', index=False, header=False)

writer.save()



C:\Users\anaol\AppData\Local\Temp\ipykernel_19452\1108826060.py:27: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
